In [ ]:
from class_PushMLFlow import PushMLFlow

In [ ]:
# Model set up libraries 
from tensorflow import keras
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Activation, ReLU
from tensorflow.keras.layers import BatchNormalization, Conv2DTranspose, Concatenate
from tensorflow.keras.models import Model, Sequential 
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers



In [ ]:
class ModelClass(PushMLFlow):
    """
    Class inerhits from PushMLFlow and will push parameters and score when run and evaluate are called.
    Inputs: X_train: [NumPy array] - trainning data
            X_test: [NumPy array] - test data
            y_train: [NumPy array] - trainning data
            y_test: [NumPy array] - test data
            loss: default='binary_crossentropy'
            experiment_name: [str] - saved name of experiemnt 
            experiment_tags: [Dictionary]{'USER': '', 'RUN NAME': '', 'VERSION':'', 'DESCRIPTION':''} - Fill in relevant meta data
    """
    

    def __init__(self, X_train, X_test, y_train, y_test, loss='binary_crossentropy',experiment_name, experiment_tags):
        super().__init__(experiment_name, experiment_tags)
        self.loss=loss

        self.X_train = X_train
        self.X_test = X_test
        self.y_train = y_train
        self.y_test = y_test

      
    def convolution_operation(self, entered_input, filters=64):  
            
        # Taking first input and implementing the first conv block
        conv1 = Conv2D(filters, kernel_size = (3,3), padding = "same")(entered_input)
        batch_norm1 = BatchNormalization()(conv1)
        act1 = ReLU()(batch_norm1)

        # Taking first input and implementing the second conv block
        conv2 = Conv2D(filters, kernel_size = (3,3), padding = "same")(act1)
        batch_norm2 = BatchNormalization()(conv2)
        act2 = ReLU()(batch_norm2)

        return act2

    def encoder(self, entered_input, filters=64):
        # Collect the start and end of each sub-block for normal pass and skip connections
        enc1 = self.convolution_operation(entered_input, filters)
        MaxPool1 = MaxPooling2D(strides = (2,2))(enc1)
        return enc1, MaxPool1

    def decoder(self, entered_input, skip, filters=64):
        # Upsampling and concatenating the essential features
        Upsample = Conv2DTranspose(filters, (2, 2), strides=2, padding="same")(entered_input)
        Connect_Skip = Concatenate()([Upsample, skip])
        out = self.convolution_operation(Connect_Skip, filters)
        return out


    def U_Net(self, Image_Size):
        # Take the image size and shape
        input1 = Input(Image_Size)

        # Construct the encoder blocks
        skip1, encoder_1 = self.encoder(input1, 64)
        skip2, encoder_2 = self.encoder(encoder_1, 64*2)
        skip3, encoder_3 = self.encoder(encoder_2, 64*4)
        skip4, encoder_4 = self.encoder(encoder_3, 64*8)

        # Preparing the next block
        conv_block = self.convolution_operation(encoder_4, 64*16)

        # Construct the decoder blocks
        decoder_1 = self.decoder(conv_block, skip4, 64*8)
        decoder_2 = self.decoder(decoder_1, skip3, 64*4)
        decoder_3 = self.decoder(decoder_2, skip2, 64*2)
        decoder_4 = self.decoder(decoder_3, skip1, 64)

        out = Conv2D(1, 1, padding="same", activation="sigmoid")(decoder_4)

        model = Model(input1, out)
        return model

    def set_model(self):
        
        # Instantiate Model
        input_shape = self.X_train.shape[1:]
        self.model = self.U_Net(input_shape)

        # Compile Model
        self.model.compile(loss=self.loss, 
                    optimizer='adam')

    def run(self):

        print(80*'-')
        print('------MODEL RUNNING------')

        # set model
        self.set_model()

        mc = ModelCheckpoint('oxford_segmentation.h5', save_best_only=True) # could put path here 

        self.model.fit(self.X_train, self.y_train, validation_split=0.3,
                  batch_size=BATCH_SIZE, epochs=EPOCHS, callbacks=[mc])

        self.mlflow_log_param('loss', self.loss)

        print(80*'=')
        print('------MODEL SUCCESFULLY------')


    def evaluate(self):
        print(80*'-')
        print('------MODEL EVALUATING------')        
        results = self.model.evaluate(self.X_test, self.y_test)
        self.mlflow_log_metric('loss', results)
        print(80*'=')
        print('------MODEL EVALUATED------')
